# Numerical Precision

In this section, you will investigate how different convolution
and matrix-matrix multiplication kernel performs when changing the
numerical precision.

## 1. Set-up

In [ ]:
# Mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Make sure your token is stored in a txt file at the location below.
# This way there is no risk that you will push it to your repo
# Never share your token with anyone, it is basically your github password!
with open('/content/gdrive/MyDrive/ece5545/token.txt') as f:
    token = f.readline().strip()
# Use another file to store your github username
with open('/content/gdrive/MyDrive/ece5545/git_username.txt') as f:
    handle = f.readline().strip()

In [ ]:
# Clone your github repo
YOUR_TOKEN = token
YOUR_HANDLE = handle
BRANCH = "main"

%mkdir /content/gdrive/MyDrive/ece5545
%cd /content/gdrive/MyDrive/ece5545
!git clone https://{YOUR_TOKEN}@github.com/ML-HW-SYS/a4-{YOUR_HANDLE}.git
%cd /content/gdrive/MyDrive/ece5545/a4-{YOUR_HANDLE}
!git checkout {BRANCH}
!git pull

PROJECT_ROOT = f"/content/gdrive/MyDrive/ece5545/a4-{YOUR_HANDLE}"

In [ ]:
# This extension reloads all imports before running each cell
%load_ext autoreload
%autoreload 2

Verify the following cell prints your github repository.

In [ ]:
!ls {PROJECT_ROOT}

In [ ]:
!pip install torch numpy matplotlib

# 2. Convolution

In the following cell(s), please plot the reconstruction error of an
approximated tensor (in the y-axis) with the numerical precision
(in the x-axis). Please show one plot for `winograd` and one plot for `fft`.

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from src.conv2d import conv2d
from src.matmul import matmul

def test_conv2d_precision(method, input_size=32, kernel_size=3, trials=100):
    """Test conv2d reconstruction error across different precisions"""
    precisions = [torch.float16, torch.float32, torch.float64]
    errors = []
    
    for precision in precisions:
        trial_errors = []
        for _ in range(trials):
            # Generate random input and kernel
            x = torch.randn(input_size, input_size, dtype=precision)
            k = torch.randn(kernel_size, kernel_size, dtype=precision)
            b = torch.randn(1, dtype=precision)
            
            # Get reference output using torch implementation
            ref = conv2d(x, k, b, method='torch')
            
            # Get output from tested method
            out = conv2d(x, k, b, method=method)
            
            # Calculate relative error
            error = torch.norm(out - ref) / torch.norm(ref)
            trial_errors.append(error.item())
            
        errors.append(np.mean(trial_errors))
    
    return precisions, errors

# Test different conv2d methods
methods = ['winograd', 'fft']
plt.figure(figsize=(10, 6))

for method in methods:
    precisions, errors = test_conv2d_precision(method)
    plt.plot([p for p in precisions], errors, 'o-', label=method)

plt.yscale('log')
plt.xlabel('Precision')
plt.ylabel('Relative Error')
plt.title('Conv2D Reconstruction Error vs Precision')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
from src.conv2d import fft
# TODO: plot the error v.s. precision curve

# 3. Matrix-matrix Multiply

In the following cell(s), please plot the reconstruction error (in the y-axis)
with the different numerical precisions (in the x-axis) for `log` (i.e.
logorithmic matrix-matrix multiplication).

In [ ]:
def test_matmul_precision(method, size=128, trials=100):
    """Test matmul reconstruction error across different precisions"""
    precisions = [torch.float16, torch.float32, torch.float64]
    errors = []
    
    for precision in precisions:
        trial_errors = []
        for _ in range(trials):
            # Generate random matrices
            A = torch.randn(size, size, dtype=precision)
            B = torch.randn(size, size, dtype=precision)
            
            # Get reference output using torch implementation
            ref = matmul(A, B, method='torch')
            
            # Get output from tested method
            out = matmul(A, B, method=method)
            
            # Calculate relative error
            error = torch.norm(out - ref) / torch.norm(ref)
            trial_errors.append(error.item())
            
        errors.append(np.mean(trial_errors))
    
    return precisions, errors

# Test different matmul methods
methods = ['log']
plt.figure(figsize=(10, 6))

for method in methods:
    precisions, errors = test_matmul_precision(method)
    plt.plot([p for p in precisions], errors, 'o-', label=method)

plt.yscale('log')
plt.xlabel('Precision')
plt.ylabel('Relative Error')
plt.title('MatMul Reconstruction Error vs Precision')
plt.legend()
plt.grid(True)
plt.show()